Added State method for parametric uncertainty, with added parameter state in control (Inverted Pendulum Example)

Objectives:
See if theory holds for control and state affine system


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
from matplotlib import MatplotlibDeprecationWarning
warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning)
import matplotlib.pyplot as plt
import jax.numpy as jnp
import jax
import numpy as np

import matplotlib
import pickle as pkl

from scipy.interpolate import interp1d
from matplotlib.animation import FuncAnimation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

import hj_reachability as hj
import utils

Setup Problem (dynamics, environment)

In [ ]:
class inv_pend(hj.ControlAndDisturbanceAffineDynamics):
    def __init__(self, uMax=10, dMin=1, dMax=2, params=None,
                 control_mode="min", disturbance_mode="max",
                 control_space=None, disturbance_space=None
                 ):
        self.uMax = jnp.array(uMax)
        self.dMin = jnp.array(dMin)
        self.dMax = jnp.array(dMax)
        if control_space is None:
            control_space = hj.sets.Box(lo=jnp.array([-self.uMax]), hi=jnp.array([self.uMax]))
        if disturbance_space is None:
            disturbance_space = hj.sets.Box(lo=jnp.array([self.dMin]), hi=jnp.array([self.dMax]))
        super().__init__(params, control_mode, disturbance_mode, control_space, disturbance_space)

    def __call__(self, state, control, disturbance, time):
        dx1 = state[1]
        dx2 = -9.81*state[3]*jnp.sin(state[0]+jnp.pi)+state[3]*control[0]
        dx3 = 0 #Parameteric dim, represented as nu
        return jnp.array([dx1, dx2, dx3])

    #Don't need to define control jacobian as there's only 1 control input
    def control_jacobian(self, state, time):
        return jnp.array([
            [0],
            [state[3]],
            [0],
        ])
    
    def disturbance_jacobian(self, state, time):
        return jnp.array([
            [0],
            [0],
            [0],
        ])
    
    def open_loop_dynamics(self, state, time):
        return jnp.array([state[1], -9.81*state[3]*jnp.sin(state[0]+jnp.pi), 0])

In [ ]:
dyn = inv_pend(uMax=10, dMin=1, dMax=1)

In [ ]:
grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(
    domain = hj.sets.Box(lo=np.array([-jnp.pi, -1, 1]), hi=np.array([jnp.pi,1, 2])),
    shape = (111, 111, 5),
    # Boundary conditions matter for accuracy on the edge of the domain grid.
    # E.g. if we have an angle theta \in [0, 2pi] then we want a periodic boundary in that dim.
    periodic_dims=0,
    #boundary_conditions=(hj.boundary_conditions.extrapolate_away_from_zero,
    #                    hj.boundary_conditions.extrapolate_away_from_zero)
)
x_init = None
direction = 'backward'